### 任务

1.主症、伴随症 -> 证

2.主症、伴随症、舌脉象 -> 证

【要求：证分多种】

#### 数据

1. 主症和伴随症

主诉症状.xlsx

2. 分型

分型数值化.xlsx

{1:'阳黄证',2:'阴黄证',3:'阴阳黄证|阴阳黄'}

#### 一、主症和伴随症的提取

使用正则匹配，1为主症，2为伴随症

In [1]:
import pandas as pd
import numpy as np
import re
import sys
sys.path.append(r'/Users/zhouzhan/Documents/codes/python_code/liver_disease/')
import constants
import utils.re_utils as re_utils

'''
    主症状和伴随症状的提取
'''

PATH = '/Users/zhouzhan/Documents/codes/python_code/liver_disease/data/症状/主诉症状.xlsx'


SYMP_DICT = {1:'肝衰竭主症', 2:'肝衰竭伴随症1|肝衰竭伴随症1持续时间量|肝衰竭伴随症2'}
def main():
    df = pd.read_excel(PATH, sheet_name='Sheet1')

    sympList = []
    # 数据过滤
    for row in df.itertuples():
        _index = getattr(row, 'Index')
        _ids = getattr(row, constants.INHOSPTIAL_ID)
        _type = getattr(row, constants.INCASE_FIELD_NAME)
        _value = getattr(row, constants.TRANSFORMED_GVALUE)

        _mathResult = re_utils.rematch(_type,SYMP_DICT)
        if _mathResult == 0: # 删除与主症，伴随症无关内容
            df.drop(index=_index,inplace=True)
            continue

        if _mathResult == 1: # 主症
            _updateValue = _value+'（主症）'
            sympList.append(_updateValue)
            df[constants.TRANSFORMED_GVALUE][_index] = _updateValue
            continue
        if _mathResult == 2: # 伴随症 
            _updateValue = _value+'（伴随）'
            sympList.append(_updateValue)
            df[constants.TRANSFORMED_GVALUE][_index] = _updateValue
            continue

    # 初始化病历症状表
    symps = list(set(sympList)) # 获得所有症状列表，作为dataframe的列
    ids = list(set(df[constants.INHOSPTIAL_ID]))    # 病人id列表，作为dataframe的行
    data = np.zeros((len(ids),len(symps)), dtype=int)   # 数据初始化，所有症状作为列，病人id作为行
    resultDF = pd.DataFrame(data=data, columns=symps, index=ids) 

    # 筛选出：主症、伴随症
    for row in df.itertuples():
        _index = getattr(row, 'Index')
        _ids = getattr(row, constants.INHOSPTIAL_ID)
        _type = getattr(row, constants.INCASE_FIELD_NAME)
        _value = getattr(row, constants.TRANSFORMED_GVALUE)

        resultDF[_value][_ids] = 1 # 记录原始症状


    resultDF.to_csv(constants.SYMP_MAIN_ACC_PATH)

if __name__ == '__main__':
    main()

<ipython-input-1-9530343a46be>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[constants.TRANSFORMED_GVALUE][_index] = _updateValue
<ipython-input-1-9530343a46be>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[constants.TRANSFORMED_GVALUE][_index] = _updateValue


#### 二、主症-伴随症-舌脉象合并


In [ ]:
# 主症-伴随症-舌脉象-分型合并
def mergeSympMainAcc_Tongue_Diagnosis():
    sympDF = pd.read_csv(constants.SYMP_MAIN_ACC_PATH)
    tongueDF = pd.read_excel(PATH_TONGUE)
    diagnosisDF = pd.read_excel(PATH_DIAGNOSIS)

    combineTongueSympDF = pd.merge(sympDF, tongueDF, how='left', on=constants.INHOSPTIAL_ID)
    resultDF = pd.merge(combineTongueSympDF, diagnosisDF, how='left', on=constants.INHOSPTIAL_ID)
    resultDF.fillna(value=0, inplace=True)

    return resultDF